In [15]:
pip install gpxcsv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 1.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from gpxcsv import gpxtolist
import pandas as pd

gpx_list = gpxtolist('/Users/BryanGutierrez/Desktop/Strava/backup/Track-11.gpx')
gpx_list
# Reads the gpx file and converts it to a list of dictionaries

gpx_df = pd.DataFrame(gpx_list)
gpx_df 
# Converts the structured data into a data frame 

,lat,lon,atemp,hr,cad,name,type,ele,time
0,39.067879,-106.118141,27.0,96.0,57.0,Morning Hike and Fishing along Rich and Rough...,hiking,3052.2,2024-07-19T15:27:54Z
1,39.067892,-106.118160,27.0,96.0,57.0,Morning Hike and Fishing along Rich and Rough...,hiking,3052.2,2024-07-19T15:27:55Z
2,39.067894,-106.118211,27.0,100.0,57.0,Morning Hike and Fishing along Rich and Rough...,hiking,3052.4,2024-07-19T15:27:58Z
3,39.067879,-106.118236,27.0,103.0,0.0,Morning Hike and Fishing along Rich and Rough...,hiking,3053.0,2024-07-19T15:28:00Z
4,39.067868,-106.118257,27.0,105.0,54.0,Morning Hike and Fishing along Rich and Rough...,hiking,3053.2,2024-07-19T15:28:02Z
...,...,...,...,...,...,...,...,...,...
4122,39.068033,-106.116381,24.0,93.0,62.0,Morning Hike and Fishing along Rich and Rough...,hiking,3041.8,2024-07-19T21:05:15Z
4123,39.068048,-106.116388,24.0,93.0,62.0,Morning Hike and Fishing along Rich and Rough...,hiking,3041.8,2024-07-19T21:05:16Z
4124,39.068118,-106.116441,24.0,91.0,0.0,Morning Hike and Fishing along Rich and Rough...,hiking,3042.4,2024-07-19T21:05:22Z
4125,39.068151,-106.116471,24.0,94.0,0.0,Morning Hike and Fishing along Rich and Rough...,hiking,3043.0,2024-07-19T21:05:25Z


In [7]:
gpx_df['time'] = pd.to_datetime(gpx_df['time'], format = '%Y-%m-%dT%H:%M:%SZ')
# Converts the time column to date time objects
# gpx_df['time'] accesses the time column
# pd.to_datetime() converts a string into datetime
# the %Y%M... corresponds to the Year-month-day... format
type(gpx_df.loc[0, 'time'])
# Uses the .loc method to access the data at index 0

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
duration = gpx_df['time'].iloc[-1] - gpx_df['time'].iloc[0]
duration
# Calculates the duration of the entire track by subtracting the first 
# timestamp from the last timestamp

Timedelta('0 days 05:37:33')

In [56]:
total_duration = duration.total_seconds()

hours = total_duration // 3600
minutes = (total_duration % 3600) // 60
# modulo returns the reaminder, it divdes by 3600 it returns the remainder, so after we divide by 3600 we have this seconds remaining and so on to find the minutes
seconds = total_duration % 60
# the remailder is converted to seconds

print(str(hours) + ' hours ' + str(minutes) + ' min ' + str(seconds) + ' sec ')

5.0 hours 37.0 min 33.0 sec 


In [9]:
import pytz
# pytz handles timezone information
from datetime import datetime
# datetime is used to work with dates and times in Python
utc_zone = pytz.utc

datetime = datetime.utcnow()

try:
    zulu_datetime = utc_zone.localize(datetime)
    formatted_datetime = zulu_datetime.strftime('%Y-%m-%d %H:%M:%S') + 'Z'
    # Formats the timezone object into year month day format
    # the + z indicates Zulu time, or 0 on the timezone map
    print("Time localized to Zulu Time:", formatted_datetime)
except Exception as e:
    print("Timezone already established:", e)


Time localized to Zulu Time: 2024-09-11 19:40:44Z


/var/folders/bf/qs8c56fs6jxbk3rgbb9k01pm0000gn/T/ipykernel_46192/2404517539.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime = datetime.utcnow()


In [3]:
pip install Timezonefinder


Note: you may need to restart the kernel to use updated packages.


In [11]:
from timezonefinder import TimezoneFinder

tf = TimezoneFinder()
# Call the method that can find the timezone of a given location

latitude = gpx_df.loc[0, 'lat']
longitude = gpx_df.loc[0, 'lon']
# Retrieve the lat and long from the dataframe
# .loc accesses the data from gpx_df in the index 0

timezone_str = tf.timezone_at(lat=latitude, lng=longitude) 
# Determines the timezone for the specific lat and lon in the dataframe

if timezone_str:
    timezone = pytz.timezone(timezone_str)
    # Pytz converts the timezone into a string
    current_time = datetime.now(timezone)
    # Gets the current datetime for the timezone
    print(f"The current local time in {timezone_str} is {current_time}")
else:
    print("Could not determine the timezone.")


The current local time in America/Denver is 2024-09-11 13:41:33.756345-06:00


5. To compute the total distance, first we would extract the coordinates, then calculate the distances between consecitive points, then we would add up all the distances between the coordinates to find the total distance traveled. This method doesn't account for changes in altitude. 